## Planning / Scratch Work

Do /r/kpop commenters talk differently about male vs. female groups?

Initial exploration of this question:
- Identify submissions on 2 all-male groups, 2 all-female groups
- Collect their comments
- Contrast comments in general to "typical" reddit language (using /r/funny as a standard)
- Contrast comments on male group vs female group 

Using Pushshift to get reddit comments

See [Pushshift's GitHub API README](https://github.com/pushshift/api)

> Search for the most recent comments mentioning the word "science" within the subreddit /r/askscience
>
> `https://api.pushshift.io/reddit/search/comment/?q=science&subreddit=askscience`

Retrieve all comment ids for a submission object

`https://api.pushshift.io/reddit/submission/comment_ids/{base36_submission_id}`

[New to Pushshift FAQ](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/)

[Pushshift Reddit API v4.0 Documentation](https://reddit-api.readthedocs.io/en/latest/#)

Not-comprehensive related works:
- "A Community of Curious Souls: An Analysis of Commenting Behavior on TED Talks Videos" (Tsou, Thelwall, Mongeon, and Sugimoto, 2014)
- "YouTube science channel video presenters and comments: female friendly or vestiges of sexism?" (Thelwall and Mas-Bleda, 2018)
- "Shirtless and dangerous: Quantifying linguistic signals of gender bias in an online fiction writing community." (Fast, Vachovsky, and Bernstein, 2016)
- "Using language models to quantify gender bias in sports journalism" (Fu, Danescu-Niculescu-Mizil, Lee, 2016)

## Data Collection

Import statements

In [7]:
import string
import re
import requests
import logging
import pickle
import json
import time

from collections import Counter

from tqdm import tqdm

import pandas as pd

import nltk
from nltk.corpus import stopwords

from data_collection_utils import *

ENGLISH_STOPWORDS = stopwords.words('english')

## Pushshift Notes

What are the values that we can access for each submission?

```python
> response.json()['data'][1].keys()

> dict_keys(['all_awardings', 'allow_live_comments', 'author', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 
'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain',
'full_link', 'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked',
'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls', 'retrieved_on', 'score', 'selftext',
'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'treatment_tags',
'upvote_ratio', 'url', 'url_overridden_by_dest', 'whitelist_status', 'wls'])
```

Note that `created_utc` is given in unix timestamp

```
> [post['created_utc'] for post in response.json()['data']]

>[1595657105,
 1595641997,
 1595632191,
 1595623051,
 1595602847,
 1595599200,
 1595583205,
 1595581926,...
```

This tells us that newer posts are given first (i.e. order of posts in repsonse.json() is newest to oldest).

What are the values that we can access for each comment?

```python
comments_json[0].keys()

dict_keys(['all_awardings', 'approved_at_utc', 'associated_award', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext',
'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
'banned_at_utc', 'body', 'can_mod_post', 'collapsed', 'collapsed_because_crowd_control', 'collapsed_reason', 'created_utc', 'distinguished', 'edited', 'gildings', 'id', 
'is_submitter', 'link_id', 'locked', 'no_follow', 'parent_id', 'permalink', 'retrieved_on', 'score', 'send_replies', 'stickied', 'subreddit', 'subreddit_id', 'top_awarded_type', 
'total_awards_received', 'treatment_tags'])
   
```


In [4]:
# # Collect 1000 posts
# posts = []
# oldest_post_utc = None
# while len(posts) < 1000:
#     if oldest_post_id is None:
#         posts = data_collection_utils.collect_posts()
#         oldest_post_utc = posts[-1]['created_utc']
#     else:
#         print('oldest_post_utc: {}'.format(oldest_post_utc))
#         time.sleep(1)
#         older_posts = data_collection_utils.collect_posts(before=oldest_post_utc)
#         oldest_post_utc = older_posts[-1]['created_utc']
#         posts.extend(older_posts)

# save_data(posts, 'data/rkpop-1000-posts.pkl')
# # test_load = load_data('rkpop-1000-posts.pkl')
# # assert test_load == posts

In [8]:
posts = load_data('data/rkpop-1000-posts.pkl')

In [9]:
post_ids = [post['id'] for post in posts]
len(set(post_ids))

1000

In [241]:
from multiprocessing import Pool
import multiprocessing

num_cpu = multiprocessing.cpu_count()

In [11]:
# Collect comments for the posts
# with Pool(10) as p:
#     comments = p.map(collect_comment, posts)

# Decided to do it serially because multiprocessing was getting my IP blocked?

comments = []
for post in tqdm(posts):
    comments.append(collect_comment(post, size_to_collect=500)) # Actually, max to collect at once is 100....

save_data(comments, 'data/rkpop-1000-comments.pkl')

100%|██████████| 1000/1000 [04:28<00:00,  3.72it/s]


In [12]:
# NONBREAKING: Rename the 500 files to 100

In [15]:
import glob

In [14]:
save_data(comments, 'data/rkpop-1000-comments.pkl')

In [25]:
# for comment_pkl in glob.glob('data/comments/*'):
#     print(len(load_data(comment_pkl))) 
# ??? Some sort f collection error... fewer than 50 for many

In [56]:
comments = {}
for filename in glob.glob('data/comments/*'):
    start = filename.rindex('/') + 1
    end = filename.rindex('-')
    post_id = filename[start:end]
    comments[post_id] = load_data(filename)

In [57]:
save_data(comments, 'data/rkpop-3000-comments.pkl')

In [44]:
comments.keys()

dict_keys(['huclsf', 'htuc60', 'hvskyu', 'hucjmo', 'hu9rcu', 'huzmb5', 'hwtrm6', 'hrp506', 'hv2356', 'ho4kpi', 'hu3597', 'huu5n2', 'huvuwp', 'hu7vo3', 'hvtdw0', 'hv1ebo', 'huzsvw', 'hqyxzz', 'hrc8b0', 'hrljep', 'hub1jr', 'hvj3uw', 'hvka7v', 'hvj615', 'hudp5x', 'htybex', 'hx29od', 'hw0msh', 'huxala', 'hudowa', 'hrp4jz', 'hv8u2z', 'hxc2tq', 'hv422q', 'hxhvbk', 'hudxp1', 'hvvai2', 'htpot9', 'hvvasx', 'hwyk30', 'hu5qlw', 'hv41za', 'humrj9', 'hx3a8s', 'hvrr9j', 'hvqe4a', 'hub3bl', 'hvqpny', 'hui3wc', 'htvhtn', 'hvvaiq', 'hvs5o0', 'huoafl', 'hudqes', 'hx9kq3', 'hv7a2v', 'hrfis5', 'hubz5p', 'hxekwy', 'hv1n7f', 'husgf7', 'humrtx', 'hujyqp', 'hwg5tm', 'hu0d49', 'hvt82o', 'htujl0', 'hr34w6', 'hv4vrd'])

In [64]:
post_ids = [p['id'] for p in posts]
post_titles = [p['title'] for p in posts]

post_ids_titles_dict = dict(zip(post_ids, post_titles))

In [59]:
comments['huclsf'][:5]

["BTS ISN'T THAT GOOD, THEY LOOK GAY",
 "I'm really just venting/ranting because people were responsible for the safety of that stage, but failed to do their job. This could be career-altering and life-changing. They ruined this year for her and potentially limited her career.\n\nLike I said, it's good that she's recovering well. We all knew from the beginning that she would recover. It's the uncertainty after recovery that I'm worried about. It just might be too troublesome continue her career after because injuries can have lasting effects on the human body long after recovery. No young person should have to deal with that.",
 "Can I just clarify that my original post meant that they got away with allowing for a dangerous set up in the first place? The injury should never have occurred and i find it really bizarre that defending SBS seems to be the hill that you're wanting to die on. I'm just expressing my frustrations that such shitty working environments are created for idols in th

In [60]:
comments['htuc60'][:5]

['definitely, i was so sad to see them not make it far :(',
 "and the best group on that show but we're not ready for that conversation yet",
 'Oh yeah, Bomin is definitely helping their group get recognized but I think a good chunk of people don’t even know he’s an idol because he’s so good at acting!',
 "Yess, my boys deserve even more success! I've been following them since their Let Me comeback and there isn't really a song I didn't like from them (but Crush is still my favorite)!\nGo GolCha!",
 "They really worked so hard for this promotion I'm so so happy to see this, they've suffered so much through 2019 because of the hiatus and then seeing them crying on that circus show made my heart break again but I'm happy that they've left those memories behind and seem happier these days. Realise how golcha has never mentioned RTK at all since leaving that shit lmao"]

In [52]:
comments.keys()

dict_keys(['huclsf', 'htuc60', 'hvskyu', 'hucjmo', 'hu9rcu', 'huzmb5', 'hwtrm6', 'hrp506', 'hv2356', 'ho4kpi', 'hu3597', 'huu5n2', 'huvuwp', 'hu7vo3', 'hvtdw0', 'hv1ebo', 'huzsvw', 'hqyxzz', 'hrc8b0', 'hrljep', 'hub1jr', 'hvj3uw', 'hvka7v', 'hvj615', 'hudp5x', 'htybex', 'hx29od', 'hw0msh', 'huxala', 'hudowa', 'hrp4jz', 'hv8u2z', 'hxc2tq', 'hv422q', 'hxhvbk', 'hudxp1', 'hvvai2', 'htpot9', 'hvvasx', 'hwyk30', 'hu5qlw', 'hv41za', 'humrj9', 'hx3a8s', 'hvrr9j', 'hvqe4a', 'hub3bl', 'hvqpny', 'hui3wc', 'htvhtn', 'hvvaiq', 'hvs5o0', 'huoafl', 'hudqes', 'hx9kq3', 'hv7a2v', 'hrfis5', 'hubz5p', 'hxekwy', 'hv1n7f', 'husgf7', 'humrtx', 'hujyqp', 'hwg5tm', 'hu0d49', 'hvt82o', 'htujl0', 'hr34w6', 'hv4vrd'])

## Loading from saved CSV

In [72]:
df.reset_index()

,index,0
0,hxhvbk,Mamamoo Wheein - Candy (orig. Baekhyun) (Speci...
1,hxekwy,Red Velvet - IRENE &amp; SEULGI - Monster (Two...
2,hxc2tq,BLACKPINK Lisa Appointed as Ambassador for BVL...
3,hx9kq3,The Rolling Stone included 9 K-Pop Boygroup so...
4,hx3a8s,PURPLE K!SS - Debut Trailer : WH0 CARES? - 유키 ...
...,...,...
95,ho4kzo,EXO-SC - On Me (Sehun Solo - Track MV)
96,ho4kpi,GFriend - Apple (MV Teaser 1)
97,hnin4k,Happy 10th Anniversary to Girl's Day!
98,hm9ctk,Happy 4th anniversary to NCT 127!


In [74]:
def get_comments_from_obj(post_id):
    if post_id in comments:
        return comments[post_id]
    else:
        return None

In [93]:
df = pd.DataFrame.from_dict(post_ids_titles_dict, orient='index')
comments_as_series = df.reset_index()['index'].apply(lambda post_id: get_comments_from_obj(post_id))
df = df.reset_index()
df['comments'] = comments_as_series
df.columns = ['id', 'title', 'comments']
df

,id,title,comments
0,hxhvbk,Mamamoo Wheein - Candy (orig. Baekhyun) (Speci...,"[I'm so happy she didn't change the pronouns, ..."
1,hxekwy,Red Velvet - IRENE &amp; SEULGI - Monster (Two...,"[RV never ever created a bad song, let alone a..."
2,hxc2tq,BLACKPINK Lisa Appointed as Ambassador for BVL...,"[count me in, what about Hera?, anyone partner..."
3,hx9kq3,The Rolling Stone included 9 K-Pop Boygroup so...,"[Lol @ the strawman, Man I'm reading through t..."
4,hx3a8s,PURPLE K!SS - Debut Trailer : WH0 CARES? - 유키 ...,"[Yes Indeed, it\`s a classic produce thing whe..."
...,...,...,...
95,ho4kzo,EXO-SC - On Me (Sehun Solo - Track MV),None
96,ho4kpi,GFriend - Apple (MV Teaser 1),"[Obvious girl detected, lol. Gfriend is for me..."
97,hnin4k,Happy 10th Anniversary to Girl's Day!,None
98,hm9ctk,Happy 4th anniversary to NCT 127!,None


In [94]:
df.to_csv('data/rkpop-data-2020-08-01.csv',index=False)

In [95]:
data_df = pd.read_csv('data/rkpop-data-2020-08-01.csv')

Identify male vs female groups

In [148]:
m_f_mapping = {'male': {'EXO', 'NCT', 'BTS', 'Stray Kids', 'G-Dragon', 'Big Bang', 
                        'AB6IX', 'Golden Child', 'SEVENTEEN', 'Top Secret', 'TST', 
                        'ONEUS', 'TVXQ', 'PENTAGON', 'THE BOYZ', 'VERIVERY', 'Ravi', 
                        'WayV', 'VIXX', 'Super Junior', 'SHINee', 'Monsta X',
                        'Block B', 'Zico', 'Treasure'},

               'female': {'GFriend', "Girl's Day", 'Red Velvet', 'AOA', 'BLACKPINK', 
               'Momoland', 'miss A', 'MAMAMOO', 'ITZY', 'Sunmi', 'Weeekly', 'NiziU', 
               'NATTY', 'Twice', 'LOONA', 'After School', 'IU', 'IZ*ONE', 'WJSN', 
               'Cosmic Girls', 'DIA', 'CHUNGHA', 'SNSD', 'Cherry Bullet', 'Somi', 
               '(G)I-DLE', 'Apink', 'Yukika', 'Oh My Girl', 'Lee Hi',
               'PURPLE K!SS', 'Singer Minty', 'Rocket Punch'}
}
m_f_mapping['male'] = {g.lower() for g in m_f_mapping['male']}
m_f_mapping['female'] = {g.lower() for g in m_f_mapping['female']}



Tag submissions with male or female

In [151]:
# Checking if any overlapping...
data_df[data_df['male'] & data_df['female']]

,id,title,comments,male,female
85,hrc2pf,"EXO-SC, MAMAMOO, Red Velvet Irene &amp; Seulgi...",NaN,True,True
